<a href="https://colab.research.google.com/github/LaraV15/TP1-AA2/blob/main/Ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este problema, se presenta un conjunto de datos que contiene información sobre el rendimiento académico de estudiantes universitarios, así como diversos factores que podrían influir en él. El objetivo es construir un modelo de regresión utilizando redes neuronales para predecir el índice de rendimiento académico de los estudiantes basado en las características proporcionadas.

## Importamos las librerías

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

In [5]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

## Análisis exploratorio

In [7]:
# Cargamos el dataset
data = pd.read_csv('Student_Performance.csv')
print(data.head())

   Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0              7               99                        Yes            9   
1              4               82                         No            4   
2              8               51                        Yes            7   
3              5               52                        Yes            5   
4              7               75                         No            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  


In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

print(train_df.head())

      Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
9254              5               49                         No            7   
1561              2               48                        Yes            7   
1670              2               81                         No            7   
6087              2               46                         No            6   
6669              8               47                         No            9   

      Sample Question Papers Practiced  Performance Index  
9254                                 5               36.0  
1561                                 6               25.0  
1670                                 2               59.0  
6087                                 1               22.0  
6669                                 0               40.0  


In [12]:
cols = train_df.columns

for col in cols:
    print("--------------------")
    print("Columna: ", col)
    print("Cantidad de valores nulos: ", train_df[col].isnull().sum())
    print("Cantidad de valores únicos: ", train_df[col].nunique())
    print("Tipo de dato: ", train_df[col].dtype)
    # Si no es numérica, continuar con la siguiente columna
    if train_df[col].dtype == "object":
        print("--------------------")
        continue
    # Si es una columna categórica, mostrar la cantidad de veces que aparece cada valor
    if train_df[col].nunique() < 10:
        print("Valores únicos: ", train_df[col].unique())
        print("Cantidad de veces que aparece cada valor: ", train_df[col].value_counts())
    else:
        # Si es una columna numérica, mostramos media, desvío estándar, mínimo, máximo...
        print("Media: ", train_df[col].mean())
        print("Desvío estándar: ", train_df[col].std())
        print("Mínimo: ", train_df[col].min())
        print("Máximo: ", train_df[col].max())
    print("--------------------")

--------------------
Columna:  Hours Studied
Cantidad de valores nulos:  0
Cantidad de valores únicos:  9
Tipo de dato:  int64
Valores únicos:  [5 2 8 3 9 4 6 1 7]
Cantidad de veces que aparece cada valor:  Hours Studied
1    937
6    917
9    894
3    891
7    889
2    871
8    869
4    869
5    863
Name: count, dtype: int64
--------------------
--------------------
Columna:  Previous Scores
Cantidad de valores nulos:  0
Cantidad de valores únicos:  60
Tipo de dato:  int64
Media:  69.531
Desvío estándar:  17.343734881577706
Mínimo:  40
Máximo:  99
--------------------
--------------------
Columna:  Extracurricular Activities
Cantidad de valores nulos:  0
Cantidad de valores únicos:  2
Tipo de dato:  object
--------------------
--------------------
Columna:  Sleep Hours
Cantidad de valores nulos:  0
Cantidad de valores únicos:  6
Tipo de dato:  int64
Valores únicos:  [7 6 9 4 5 8]
Cantidad de veces que aparece cada valor:  Sleep Hours
8    1437
6    1341
9    1327
7    1326
5    1290
4

In [14]:
# Defino un diccionario para definir el tipo de dato de cada columna

column_types = {
    "Hours Studied": "numeric",
    "Previous Scores": "numeric",
    "Extracurricular Activities": "categoric",
    "Sleep Hours": "numeric",
    "Sample Question Papers Practiced": "numeric",
}

# Diccionario para almacenar media y desvío estándar de las columnas numéricas.
# Me va a servir para realizar predicciones en el futuro.
numeric_stats = {}

# Diccionario para almacenar los valores únicos de las columnas categóricas.
# Me va a servir para realizar predicciones en el futuro.
categoric_values = {}

preprocessed_train_df = pd.DataFrame()
preprocessed_test_df = pd.DataFrame()

# Preprocesamiento del set de entrenamiento
for col in column_types.keys():
    if column_types[col] == "categoric":
        num_classes = train_df[col].nunique()
        # Obtengo el one-hot encoding de la columna
        one_hot = pd.get_dummies(train_df[col], prefix=col, dtype=np.float32)
        # Agrego las columnas al dataset preprocesado
        preprocessed_train_df = pd.concat([preprocessed_train_df, one_hot], axis=1)
        # Almaceno los valores únicos
        categoric_values[col] = train_df[col].unique()
    else:
        # Normalizo la columna
        preprocessed_train_df[col] = (train_df[col] - train_df[col].mean()) / train_df[col].std()
        # Almaceno media y desvío estándar
        numeric_stats[col] = {
            "mean": train_df[col].mean(),
            "std": train_df[col].std()
        }

# Agrergo la columna "Performance Index" al dataset preprocesado
preprocessed_train_df["Performance Index"] = train_df["Performance Index"]

# Muestro las primeras filas del dataset preprocesado
print(preprocessed_train_df.head())

# Drop nan values
preprocessed_train_df = preprocessed_train_df.dropna()

# Defino los vectores X_train, y_train, X_test e y_test
X_train = preprocessed_train_df.drop("Performance Index", axis=1).values.astype(np.float32)
y_train = preprocessed_train_df["Performance Index"].values.astype(np.float32)

print(X_train.shape, y_train.shape)


      Hours Studied  Previous Scores  Extracurricular Activities_No  \
9254       0.006455        -1.183770                            1.0   
1561      -1.149676        -1.241428                            0.0   
1670      -1.149676         0.661276                            1.0   
6087      -1.149676        -1.356744                            1.0   
6669       1.162586        -1.299086                            1.0   

      Extracurricular Activities_Yes  Sleep Hours  \
9254                             0.0     0.269872   
1561                             1.0     0.269872   
1670                             0.0     0.269872   
6087                             0.0    -0.318886   
6669                             0.0     1.447386   

      Sample Question Papers Practiced  Performance Index  
9254                          0.134033               36.0  
1561                          0.483531               25.0  
1670                         -0.914463               59.0  
6087          

In [17]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,073 (12.00 KB)

 Trainable params: 3,073 (12.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0000e+00 - loss: -1081.0416 - val_accuracy: 0.0000e+00 - val_loss: -31788.3848
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -118537.5938 - val_accuracy: 0.0000e+00 - val_loss: -744719.6250
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -1361558.2500 - val_accuracy: 0.0000e+00 - val_loss: -4238775.0000
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -6345863.0000 - val_accuracy: 0.0000e+00 - val_loss: -13775702.0000
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -18221920.0000 - val_accuracy: 0.0000e+00 - val_loss: -33332372.0000
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -41476856.0000 - val_accuracy: 0.0000e+00 - val_loss: -66978000.0000
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -79892640.00